In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.neighbors import LocalOutlierFactor 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [9]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [10]:
scaler = StandardScaler()
X.head

<bound method NDFrame.head of      SESSO  ANNI  PESO  ALTEZZA        BMI  Mezzo  Testa:Neurocranio  \
0      0.0  81.0  84.0    175.0  27.428571    0.0                1.0   
1      1.0  69.0  69.0    162.0  26.291724    1.0                4.0   
2      1.0  71.0  67.0    155.0  27.887617    1.0                2.0   
3      1.0  54.0  60.0    159.0  23.733238    1.0                4.0   
4      1.0  78.0  69.0    167.0  24.740937    1.0                2.0   
..     ...   ...   ...      ...        ...    ...                ...   
125    1.0  82.0  80.0    162.0  30.559366    0.0                1.0   
126    1.0  70.0  75.0    157.0  30.670615    0.0                2.0   
127    0.0  45.0  45.0    177.0  14.363689    1.0                0.0   
128    0.0  54.0  89.0    168.0  31.746032    0.0                0.0   
129    1.0  86.0  50.0    155.0  20.811655    0.0                1.0   

     Testa:Splancnocranio  Testa:Telencefalo  Testa:Cervelletto  ...  \
0                     0.0        

In [11]:
X_scaled = scaler.fit_transform(X)
X_scaled.shape

(130, 274)

In [12]:
X_scaled[0]

array([-0.82964552,  0.89567642,  0.75569732,  0.95952823,  0.19233301,
       -0.9258201 , -0.39545008, -0.55043459, -0.73989059, -0.38333515,
        0.29923494, -0.92543669, -0.32422414, -0.57087214, -0.50792681,
       -0.38905465,  1.29119578, -0.4601548 , -0.17696113, -0.31158516,
       -0.31950598, -0.41404911,  1.57221493, -0.19505392,  1.24281454,
        0.07868895, -0.48795004, -0.76509206, -0.48795004, -0.46363053,
       -0.72760688, -0.48795004, -0.55958137, -0.94024357, -0.63075309,
       -0.57142857, -0.84292723,  1.55597321, -0.41372838, -0.45133547,
       -0.40089186, -0.42640143, -0.54772256, -0.21997067, -0.64046332,
       -0.23855936, -0.3188964 , -0.2       , -0.15369466, -0.15369466,
       -0.17817416, -0.15369466, -0.08804509, -0.17817416, -0.23855936,
       -0.28867513,  3.90512484, -0.23855936, -0.3188964 , -0.2       ,
       -0.2       , -0.21997067, -0.23855936, -0.21997067, -0.2       ,
       -0.08804509, -0.125     , -0.37109635, -0.3188964 , -0.08

In [14]:
X.values[0]

array([  0.        ,  81.        ,  84.        , 175.        ,
        27.42857143,   0.        ,   1.        ,   0.        ,
         0.        ,   0.        ,   1.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         3.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   3.        ,   0.        ,
         3.        ,   3.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   1.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [32]:
def nested_cv_accuracy(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_neighborss, contaminations, metrics):
                    clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['n_neighbors'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [33]:
results = nested_cv_accuracy(X)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.5,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.49703,52.428571,16.200907,"no const, no equals, no tot, standard scaled"


In [34]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [35]:
def nested_cv_f1(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_neighborss, contaminations, metrics):
                    clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['n_neighbors'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [36]:
scores_df = add_record(scores_df, nested_cv_f1(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"


In [37]:
def outer_cv_inner_holdout_accuracy(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [38]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_accuracy(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"


In [39]:
def outer_cv_inner_holdout_f1(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [40]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_f1(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"
3,LocalOutlierFactor,7,0.47,manhattan,f1 on outer cv inner holdout,63.230769,7.850677,67.966572,10.216902,64.000000,15.905332,"no const, no equals, no tot, standard scaled"


In [41]:
def outer_holdout_inner_cv_accuracy(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0
        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [42]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_accuracy(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"
3,LocalOutlierFactor,7,0.47,manhattan,f1 on outer cv inner holdout,63.230769,7.850677,67.966572,10.216902,64.000000,15.905332,"no const, no equals, no tot, standard scaled"
4,LocalOutlierFactor,7,0.49,manhattan,accuracy on outer holdout inner cv,66.153846,8.904490,70.704906,11.723844,57.817051,15.822948,"no const, no equals, no tot, standard scaled"


In [43]:
def outer_holdout_inner_cv_f1(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [44]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_f1(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"
3,LocalOutlierFactor,7,0.47,manhattan,f1 on outer cv inner holdout,63.230769,7.850677,67.966572,10.216902,64.000000,15.905332,"no const, no equals, no tot, standard scaled"
4,LocalOutlierFactor,7,0.49,manhattan,accuracy on outer holdout inner cv,66.153846,8.904490,70.704906,11.723844,57.817051,15.822948,"no const, no equals, no tot, standard scaled"
5,LocalOutlierFactor,30,0.46,cosine,f1 on outer holdout inner cv,61.153846,4.015503,68.264069,8.028956,53.795330,10.196983,"no const, no equals, no tot, standard scaled"


In [45]:
def double_holdout_accuracy(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_neighborss, contaminations, metrics):
            clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['n_neighbors'] = params[0]
                best_params['contamination'] = params[1]
                best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [46]:
scores_df = add_record(scores_df, double_holdout_accuracy(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"
3,LocalOutlierFactor,7,0.47,manhattan,f1 on outer cv inner holdout,63.230769,7.850677,67.966572,10.216902,64.000000,15.905332,"no const, no equals, no tot, standard scaled"
4,LocalOutlierFactor,7,0.49,manhattan,accuracy on outer holdout inner cv,66.153846,8.904490,70.704906,11.723844,57.817051,15.822948,"no const, no equals, no tot, standard scaled"
5,LocalOutlierFactor,30,0.46,cosine,f1 on outer holdout inner cv,61.153846,4.015503,68.264069,8.028956,53.795330,10.196983,"no const, no equals, no tot, standard scaled"
6,LocalOutlierFactor,5,0.48,manhattan,accuracy on double holdout,61.923077,7.585801,69.615801,13.842357,53.097356,14.053180,"no const, no equals, no tot, standard scaled"


In [47]:
def double_holdout_f1(X):
    n_neighborss = [3, 5, 7, 10, 15, 20, 25, 30]
    contaminations = np.linspace(0.01, 0.5, 50)
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_neighborss, contaminations, metrics):
            clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['n_neighbors'] = params[0]
                best_params['contamination'] = params[1]
                best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [48]:
scores_df = add_record(scores_df, double_holdout_f1(X))
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,7,0.50,manhattan,accuracy on nested cv,60.153846,11.532306,67.331478,13.497030,52.428571,16.200907,"no const, no equals, no tot, standard scaled"
1,LocalOutlierFactor,15,0.34,cosine,f1 on nested cv,62.769231,10.643219,67.056093,11.067875,63.000000,17.710253,"no const, no equals, no tot, standard scaled"
2,LocalOutlierFactor,5,0.50,manhattan,accuracy on outer cv inner holdout,63.846154,9.069097,73.628061,10.973166,52.285714,16.869245,"no const, no equals, no tot, standard scaled"
3,LocalOutlierFactor,7,0.47,manhattan,f1 on outer cv inner holdout,63.230769,7.850677,67.966572,10.216902,64.000000,15.905332,"no const, no equals, no tot, standard scaled"
4,LocalOutlierFactor,7,0.49,manhattan,accuracy on outer holdout inner cv,66.153846,8.904490,70.704906,11.723844,57.817051,15.822948,"no const, no equals, no tot, standard scaled"
5,LocalOutlierFactor,30,0.46,cosine,f1 on outer holdout inner cv,61.153846,4.015503,68.264069,8.028956,53.795330,10.196983,"no const, no equals, no tot, standard scaled"
6,LocalOutlierFactor,5,0.48,manhattan,accuracy on double holdout,61.923077,7.585801,69.615801,13.842357,53.097356,14.053180,"no const, no equals, no tot, standard scaled"
7,LocalOutlierFactor,25,0.49,cosine,f1 on double holdout,65.769231,10.384615,76.678516,12.512746,59.637141,14.271097,"no const, no equals, no tot, standard scaled"


In [49]:
file_path = "lof_exp3_df.pickle"

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)